In [5]:
from bs4 import BeautifulSoup
import requests
import csv
import os
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import multiprocessing

In [6]:
html_parser = {
    'title': 'art-title',
    'department': 'art-product-name',
    'author': 'art-author',
    'date': 'art-datetime',
    'text': 'art_paragraph'
    }

def get_articles(url):
    soup = BeautifulSoup(requests.get(url).text, "html.parser")
    articles = soup.find_all(class_='result-item-link',href=True)
    return articles

def get_article_data(link, driver):
    data = {
        'title': None,
        'department': None,
        'author': None,
        'date': None,
        'text': None,
        'link': link
    }
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    for key, value in html_parser.items():
        try:
            if key == 'text':
                paragraphs = soup.find_all(class_=value)
                data[key] = " ".join([paragraph.text for  paragraph in paragraphs]).replace('\n',"").strip()
            else:
                data[key] = soup.find(class_=value).text.replace('\n',"").strip()
        except:
            print(f"Key: {key} not found in {link}")
    return data

def scrap_page(url):
    chrome_options = Options()
    chrome_options.add_argument("user-data-dir=selenium")
    chrome_options.add_argument('headless')
    driver = webdriver.Chrome(options=chrome_options)
    articles = get_articles(url)
    data = []
    for article in articles:
        link='https://classic.wyborcza.pl/archiwumGW/'+article['href']
        data.append(get_article_data(link, driver))
    driver.close()
    return data

def valdidate_csv(csv_file):
    if not os.path.exists(csv_file):
        print(f"Creating file {csv_file}")
        column_names = ['Title', 'Department', 'Author', 'Date', 'Text', 'Link']
        with open(csv_file, 'w') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(column_names)
    else:
        print(f"Appending to {csv_file}")

In [7]:
csv_file = 'wyborcza.csv'
valdidate_csv('wyborcza.csv')

Appending to wyborcza.csv


In [8]:
def process_page(page):
    link_page = f'https://classic.wyborcza.pl/archiwumGW/0,160510.html?searchForm=&datePeriod=0&initDate=2015-01-01&endDate=2023-01-01&publicationsString=1%3B5&author=&page={page}&sort=OLDEST'
    data_list = scrap_page(link_page)
    print(f"Page {page}")
    return data_list

with multiprocessing.Pool(4) as pool:
    results = pool.map(process_page, range(10))

with open(csv_file, 'a', newline='') as file:
    writer = csv.writer(file)
    for data_list in results:
        for data in data_list:
            writer.writerow([data['title'], data['department'], data['author'], data['date'], data['text'], data['link']])

Key: author not found in https://classic.wyborcza.pl/archiwumGW/7971837/2014---NASZE-NAJLEPSZE-ZDJECIA
Page 4
Page 5
Page 7
Page 6
Page 9


WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: chrome-headless-shell=120.0.6099.129)
Stacktrace:
#0 0x55c5395b9f83 <unknown>
#1 0x55c539272b2b <unknown>
#2 0x55c53925a16d <unknown>
#3 0x55c539259882 <unknown>
#4 0x55c539258586 <unknown>
#5 0x55c53925844a <unknown>
#6 0x55c5392567e1 <unknown>
#7 0x55c53925718a <unknown>
#8 0x55c53926807c <unknown>
#9 0x55c5392807c1 <unknown>
#10 0x55c5392866bb <unknown>
#11 0x55c53925792d <unknown>
#12 0x55c539280459 <unknown>
#13 0x55c53930af1a <unknown>
#14 0x55c5392ebe53 <unknown>
#15 0x55c5392b3dd4 <unknown>
#16 0x55c5392b51de <unknown>
#17 0x55c53957e531 <unknown>
#18 0x55c539582455 <unknown>
#19 0x55c53956af55 <unknown>
#20 0x55c5395830ef <unknown>
#21 0x55c53954e99f <unknown>
#22 0x55c5395a7008 <unknown>
#23 0x55c5395a71d7 <unknown>
#24 0x55c5395b9124 <unknown>
#25 0x7f4b9ee61947 start_thread
#26 0x7f4b9eee7860 __clone3
